![image.png](attachment:image.png)

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [1]:
# !pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 199 kB 55.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=97e8a57bea44ff3b88d3084087f88aeb22fbf0facee4b49465ba3a521235c44a
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [1]:
import pyspark
from pyspark.sql.functions import count,avg,sum,mean
# from pyspark.sql.types import TimestampType
from pyspark.sql.functions import col, isnan, when,udf,datediff, isnull, log

In [2]:
import pandas as pd
import numpy as np


In [3]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import Tokenizer, RegexTokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes


In [4]:
from IPython.display import display, HTML
display(HTML("<style>pre {white-space : pre !important ;}</style>"))

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [6]:
df = spark.read.option("header", "false") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv("/content/SMSSpamCollection.txt") \
    .withColumnRenamed("_c0", "class") \
    .withColumnRenamed("_c1", "text")

### Print the schema

In [7]:
df.printSchema()


root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Rename the first column to 'class' and second column to 'text'

In [7]:
## did that on Reading the Dataframe step above

### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [8]:
df.limit(10).show(truncate=True)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+



In [9]:
df.limit(10).show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [10]:
from pyspark.sql.functions import length
df_2 = df.withColumn('length',length(df['text']))

### Show the new dataframe

In [11]:
df_2.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



### Get the average text length for each class (give alias name to the average length column)

In [12]:
df_2.groupby('class').agg(mean("length").alias("Avg_Lenghth")).show()

+-----+-----------------+
|class|      Avg_Lenghth|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [13]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")


In [14]:
stop_words_remover = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')


In [15]:
count_Vectorizer = CountVectorizer(inputCol='stop_tokens',outputCol='count_vec')


In [16]:
TF_IDF = IDF(inputCol="count_vec", outputCol="tf_idf")


- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [17]:
String_Indexer = StringIndexer(inputCol='class',outputCol='label')


In [18]:
Vector_Assembler = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')


## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [19]:
nb = NaiveBayes()


## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [20]:
from pyspark.ml import Pipeline
Pipe_Line = Pipeline(stages=[String_Indexer,tokenizer,stop_words_remover,count_Vectorizer,TF_IDF,Vector_Assembler,nb])


### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [22]:
(training,testing) = df_2.randomSplit([0.7,0.3])


In [23]:
training.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham| &lt;#&gt;  mins ...|    51|
|  ham| &lt;DECIMAL&gt; ...|   132|
|  ham| and  picking the...|    41|
|  ham| came to look at ...|   103|
|  ham| gonna let me kno...|    95|
|  ham| said kiss, kiss,...|   133|
|  ham| says that he's q...|   200|
|  ham|"Gimme a few" was...|    41|
|  ham|"Happy valentines...|   147|
|  ham|"Life is nothing ...|   159|
|  ham|"The world suffer...|   129|
|  ham|"Wen u miss someo...|   143|
|  ham|&lt;#&gt;  am I t...|    45|
|  ham|&lt;#&gt;  great ...|    85|
|  ham|&lt;#&gt;  is fas...|   461|
|  ham|&lt;#&gt;  w jett...|    37|
|  ham|&lt;#&gt; %of ppl...|   327|
|  ham|&lt;#&gt; , that'...|    48|
|  ham|'An Amazing Quote...|   141|
|  ham|(I should add tha...|   132|
+-----+--------------------+------+
only showing top 20 rows



### Fit your Pipeline model to the training data

In [24]:
Predictor_fit = Pipe_Line.fit(training)


### Perform predictions on tests dataframe

In [25]:
Predictor_Results = Predictor_fit.transform(testing)


In [26]:
Predictor_Results.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|class|                text|length|label|          token_text|         stop_tokens|           count_vec|              tf_idf|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  ham| &lt;#&gt;  in mc...|    36|  0.0|[, &lt;#&gt;, , i...|[, &lt;#&gt;, , m...|(10781,[3,6,8091]...|(10781,[3,6,8091]...|(10782,[3,6,8091,...|[-159.79317525429...|[1.0,1.3157285759...|       0.0|
|  ham| what number do u...|    36|  0.0|[, what, number, ...|[, number, u, liv...|(10781,[0,3,71,20...|(10781,[0,3,71,20...|(10782,[0,3,71,20...|[-131.77352585091...|[0.99995836005344...|       0.0|


### Print the schema of the prediction dataframe

In [27]:
Predictor_Results.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- label: double (nullable = false)
 |-- token_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- count_vec: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [28]:
# from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [29]:
Evaluator = MulticlassClassificationEvaluator()
f1_score = Evaluator.evaluate(Predictor_Results)


In [30]:
print ("f1_score is: ", f1_score)


f1_score is:  0.9789991382826017


f1_score is: 0.9664707489549014


# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>

![image-3.png](attachment:image-3.png)